Limpieza de datos

In [27]:
# Importamos la librería pandas para el manejo y análisis de datos
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#db_AGP_2019_sucesos.cvs : Hace referencia al detalle de la publicacion y las personas que comentaron. 
df_publicaciones = pd.read_csv("datasets/db_AGP_2019_sucesos.csv", sep=",")

#postID_suceso.cvs                   : Hace referencia al ID de la publicacion y el suceso que hubo.
df_sucesos = pd.read_csv("datasets/postID_suceso.csv", sep=",")
df_publicaciones["object_link.connections.comments.message"].value_counts()

object_link.connections.comments.message
Cobarde                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [28]:
# Suponiendo que tu DataFrame se llama df y la columna es 'FECHA Y HORA'
df_publicaciones['FECHA'] = df_publicaciones['FECHA Y HORA'].apply(lambda x: x.split()[0])
df_publicaciones['HORA'] = df_publicaciones['created_time.2'].apply(lambda x: x.split()[1])
df_publicaciones = df_publicaciones.drop(['created_time.1','created_time.2','FECHA Y HORA'], axis=1)

In [29]:
df_publicaciones.head()

,message,id,object_link.connections.comments.message,FECHA,HORA
0,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🧐🧐,19/04/2019,09:16:15
1,"Exequias del ex presidente Alan García en ""La ...",71263708835_276236269920595,🥺🥺🥺🥺🥺🥺🥺🥺,18/04/2019,20:57:16
2,🚨 Estamos en los exteriores del Hospital de Em...,71263708835_310239669653341,🥳,17/04/2019,07:37:30
3,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🤷‍♀️🤷‍♀️🤷‍♀️,19/04/2019,09:16:15
4,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🤯🤯🤯🤯,19/04/2019,09:16:15


In [30]:
#Veamos los nulos y duplicados
df_publicaciones.isna().sum()#no hay nulos ni duplicados en la tabla de Publicaciones
df_publicaciones.duplicated().sum()
df_sucesos.isna().sum()#no hay nulos ni dulicados en la tabla de Sucesos
df_sucesos.duplicated().sum()

0

In [31]:
#estandarizacion de la nomencaltura de los encabezados de los dos dataframes
df_publicaciones = df_publicaciones.rename(columns={
    'message': 'Publicacion',
    'id': 'Id_posteo',
    'object_link.connections.comments.message': 'Comentario',
    'FECHA': 'Fecha',
    'HORA': 'Hora'
})

df_sucesos = df_sucesos.rename(columns={
    'POST NRO': 'Id_posteo',
    'SUCESO': 'Suceso'
})

In [32]:
#tipo de datos a cada campo de los dfs
df_publicaciones = df_publicaciones.rename(columns={
    'message': 'Publicacion',
    'id': 'Id_posteo',
    'object_link.connections.comments.message': 'Comentario',
    'FECHA': 'Fecha',
    'HORA': 'Hora'
})

# Asegurar que las columnas de texto sean de tipo string
df_publicaciones["Publicacion"] = df_publicaciones["Publicacion"].astype("string")
df_publicaciones["Id_posteo"] = df_publicaciones["Id_posteo"].astype("string")
df_publicaciones["Comentario"] = df_publicaciones["Comentario"].astype("string")

# Convertir Fecha y Hora
df_publicaciones["Fecha"] = pd.to_datetime(df_publicaciones["Fecha"], dayfirst=True, errors="coerce")  # Maneja valores no convertibles con NaT
df_publicaciones["Hora"] = pd.to_datetime(df_publicaciones["Hora"], format="%H:%M:%S", errors="coerce").dt.time  # Extrae la parte de hora

# Verificar los tipos de datos
print(df_publicaciones.dtypes)
# print(type(df["Hora"].iloc[0]))
df_sucesos["Id_posteo"] = df_sucesos["Id_posteo"].astype("string")
df_sucesos["Suceso"] = df_sucesos["Suceso"].astype("string")

print(df_sucesos.dtypes)
df_sucesos.Suceso.value_counts()

Publicacion    string[python]
Id_posteo      string[python]
Comentario     string[python]
Fecha          datetime64[ns]
Hora                   object
dtype: object
Id_posteo    string[python]
Suceso       string[python]
dtype: object


Suceso
Confirmación de Muerte                 28
Velatorio Casa del Pueblo              22
Lectura de Carta de AG                 11
Ataud Sale del local APRA              11
Portadas del día                        9
Confirmación de Intento de Suicidio     5
Traslado a la Clinica                   1
Name: count, dtype: Int64

In [33]:
# Veamos como esta distribuido la frecuencia de palabras de los comentarios
df_publicaciones.Comentario.value_counts()

Comentario
Cobarde                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Se visualiza que los comentarios están en mayúsculas y minusculas, además que muchos cuentan con emojis, etiquetas y tildes. El siguiente pase es normalizar esta data.

3) Normalización

In [34]:
# Importamos las librerias necesaria para la normalización de los comentarios
import re
import nltk
from nltk.corpus import stopwords # Libreria que nos ayudara a limpiar palabras conocidas como stopwords
import unicodedata
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eliza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
# Creamos una función para primero eliminar acentos o tildes
def remover_acentos(text):
    text = unicodedata.normalize('NFKD', text)
    return ''.join([c for c in text if not unicodedata.combining(c)])

# Aplicar al texto limpio
df_publicaciones['Comentario_limpio'] = df_publicaciones['Comentario'].apply(remover_acentos)

# Visualizamos el resultado
print(df_publicaciones.Comentario_limpio.value_counts().head())

Comentario_limpio
Cobarde                   135
Abran el cajon            115
Cortina de humo            52
Esta vivo                  47
EL APRA NUNCA MUERE...     41
Name: count, dtype: int64


In [36]:
# Ahora creamos una función para normalizar los comentarios:
def clean_text(text):
    # Convertir a string y minúsculas
    text = str(text).lower()
    # Eliminamos URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) 
    # http\S+: Busca cadenas que comiencen con http seguido de uno 
    # o más caracteres que no sean espacios (\S significa "no espacio",
    # y + indica "uno o más").
    
    # Eliminamos menciones (@usuario) y hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    # Eliminamos caracteres especiales y números
    text = re.sub(r'[^A-Za-záéíóúñü\s]', '', text)
    # Tokenizamos el texto y eliminamos stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    # Reconstruimos el texto limpio
    cleaned_text = ' '.join(filtered_words)

    return cleaned_text

In [37]:
# Aplicamos la función a nuestro dataset
df_publicaciones['Comentario_limpio'] = df_publicaciones['Comentario_limpio'].apply(clean_text)
# Visualizamos el resultado
df_publicaciones['Comentario_limpio'].value_counts()

Comentario_limpio
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 740
cobarde                                                                                                                                                                                                                                                                                                                          

In [38]:
# Diccionario de palabras mal escritas o similares y sus correcciones
replacement_dict = {
    "habran": "abran",
    "ataud": "cajon",
    'atau':'cajon',
    'abren':'abran',
    'caja': 'cajon',
    'muerte': 'muerto',
    'fallecio':'muerto',
    'murio':'muerto',
    'muera':'muerto',
    'farsa':'falso',
    'falsa':'falso',
    'FALSO':'falso',
    'aprietas': 'apristas',
    'feretro':'cajon',
    'cajonnnn':'cajon',
    'canjon': 'cajon',
    'ver':'verlo',
    'verlooooooooo':'verlo',
    'vea':'verlo',
    'vio':'verlo',
    'cobardia':'cobarde',
    'vean':'verlo',
    'veran':'verlo',
    'cadaver':'cuerpo',
    'fuga':'fugo',
    'escapo':'fugo',
    'fugar':'fugo',
    'escapa':'fugo',
    'morir':'muerto',
    'cara':'rostro',
    'mostralo':'mostrarlo',
    'ataudque':'cajon',
    'fugan':'fugo',
    'aban':'abran',
    'abarn':'abran',
    'abiero':'abran',
    'ablan':'abran',
    'cagon':'cajon',
    'abre':'abran',
    'corruptos':'corrupto'
}

# Creamos una función para reemplazar las palabras seleccionadas
def replace_words(text, replacements):
    words = text.split()
    corrected_words = [replacements.get(word, word) for word in words]
    return " ".join(corrected_words)

# Aplicamos el texto normalizado
df_publicaciones['Comentario_limpio'] = df_publicaciones['Comentario_limpio'].apply(lambda x: replace_words(x, replacement_dict))

In [39]:
# Visualizamos el resultado
print(df_publicaciones['Comentario_limpio'].value_counts())

Comentario_limpio
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 740
abran cajon                                                                                                                                                                                                                                                                                                                      

In [40]:
# Reemplazar espacios en blanco por NaN
df_publicaciones.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [41]:
# Eliminamos los valores que quedaron nulos
df_publicaciones.dropna(inplace=True)

In [42]:
def normalize_abran(text):
    # Patrón para identificar palabras similares a "abran"
    pattern = r'\babran\w*'
    # Reemplazar todas las coincidencias con "abran"
    normalized_text = re.sub(pattern, 'abran', text, flags=re.IGNORECASE)
    return normalized_text
df_publicaciones['Comentario_limpio'] = df_publicaciones['Comentario_limpio'].apply(normalize_abran)

In [43]:
df_publicaciones['Comentario_limpio'].value_counts().head(10)

Comentario_limpio
abran cajon                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [44]:
# Volvemos a revisar que no haya valores nulos
df_publicaciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54266 entries, 20 to 55005
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Publicacion        54266 non-null  string        
 1   Id_posteo          54266 non-null  string        
 2   Comentario         54266 non-null  string        
 3   Fecha              54266 non-null  datetime64[ns]
 4   Hora               54266 non-null  object        
 5   Comentario_limpio  54266 non-null  object        
dtypes: datetime64[ns](1), object(2), string(3)
memory usage: 2.9+ MB


In [45]:
# Vemos como va quedando el dataset
df_publicaciones.head()

,Publicacion,Id_posteo,Comentario,Fecha,Hora,Comentario_limpio
20,El congresista aprista Jorge del Castillo seña...,71263708835_10157690605563836,🤭🤭🤭🤭 Estoy trabajando ese señor no me Pagará p...,2019-04-17,16:21:20,trabajando senor pagara hacer duelo
24,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🤨🤨🤨🤨.....mmmmmm,2019-04-19,09:16:15,mmmmmm
29,🔴 #AHORA | El ex mandatario Ollanta Humala lle...,71263708835_10157694133878836,🤦🏻‍♂️🤦🏻‍♂️🤦🏻‍♂️🤦🏻‍♂️🤦🏻‍♂️ aprietas de mierda,2019-04-18,17:22:20,apristas mierda
31,EN VIVO | Así se desarrolla el velatorio del e...,71263708835_1509825649150231,🤦🏻‍♀️🤦🏻‍♀️🤦🏻‍♀️ realmente un circo,2019-04-18,10:36:35,realmente circo
35,"Decenas de simpatizantes del Partido Aprista, ...",71263708835_10157691315508836,🤦‍♂️ Parfavar....... Que gente más figureti. S...,2019-04-18,19:12:45,parfavar gente mas figureti si extranaran sera...


4) Creamos nuestro modelo de análisis de sentimientos

In [46]:
!pip install vaderSentiment


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\eliza\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [47]:
#Realizamos un análisis de sentimiento
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Definimos nuestro modelo
analyzer = SentimentIntensityAnalyzer()

# Creamos una función para detectar que emoticones y mensajes tomaria como sarcasmo o ironia
def detectar_sarcasmo(texto):
    marcadores_sarcasmo = [
        '🎼','🐀','🥱','😂','🤦🏻‍♀️','🤭','🤦🏻‍♂️','🤦‍♀️','😏', 'claro', 'obviamente', 'como si', "jajaja", "jaja",
        "🙄", "😏", "🤔", "🤨", "🙃", "🤣", "😹", "😆","😜", "😝", "🤪", "🥴","🤷", "😒", "😑", "😬",
        "👌", "💅", "🤡", "😎", "👏", "🥱","🤦", "🤦‍♂️", "🤦‍♀️", "🤷‍♂️", "🤷‍♀️", "🥹","claro que sí", "qué original",
        "buenísimo", "perfecto","me encanta", "excelente", "fabuloso", "maravilloso","brillante", "asombroso",
        "por supuesto", "cómo no", "seguro", "genial", "fantástico", "qué sorpresa", "qué emoción","impresionante",
        "todo bien","gracias por nada", "era justo lo que necesitaba","qué conveniente", "qué buena idea", "sí",
        "sin comentarios", "no esperaba menos", "todo un éxito"
        ]
    texto = texto.lower()
    if any(marcador in texto for marcador in marcadores_sarcasmo):
        return "sarcasmo"
    return "normal"

# Creamos otra función combinada para clasificar sentimiento y sarcasmo
def clasificar_sentimiento_con_sarcasmo(texto):
    sarcasmo = detectar_sarcasmo(texto)
    if sarcasmo == "sarcasmo":
        return "sarcasmo"
    else:
        puntaje = analyzer.polarity_scores(texto)
        if puntaje['compound'] > 0.05:
            return "positivo"
        elif puntaje['compound'] < -0.05:
            return "negativo"
        else:
            return "neutral"

# Aplicamos la función a nuestro dataset
df_publicaciones['clasificacion'] = df_publicaciones['Comentario'].apply(clasificar_sentimiento_con_sarcasmo)

In [48]:
# Vemos por última vez como va quedando nuestro dataset
df_publicaciones.head(70)

,Publicacion,Id_posteo,Comentario,Fecha,Hora,Comentario_limpio,clasificacion
20,El congresista aprista Jorge del Castillo seña...,71263708835_10157690605563836,🤭🤭🤭🤭 Estoy trabajando ese señor no me Pagará p...,2019-04-17,16:21:20,trabajando senor pagara hacer duelo,sarcasmo
24,Velorio del ex presidente Alan García en la Ca...,71263708835_377868469727477,🤨🤨🤨🤨.....mmmmmm,2019-04-19,09:16:15,mmmmmm,sarcasmo
29,🔴 #AHORA | El ex mandatario Ollanta Humala lle...,71263708835_10157694133878836,🤦🏻‍♂️🤦🏻‍♂️🤦🏻‍♂️🤦🏻‍♂️🤦🏻‍♂️ aprietas de mierda,2019-04-18,17:22:20,apristas mierda,sarcasmo
31,EN VIVO | Así se desarrolla el velatorio del e...,71263708835_1509825649150231,🤦🏻‍♀️🤦🏻‍♀️🤦🏻‍♀️ realmente un circo,2019-04-18,10:36:35,realmente circo,sarcasmo
35,"Decenas de simpatizantes del Partido Aprista, ...",71263708835_10157691315508836,🤦‍♂️ Parfavar....... Que gente más figureti. S...,2019-04-18,19:12:45,parfavar gente mas figureti si extranaran sera...,sarcasmo
...,...,...,...,...,...,...,...
358,Retiran los restos de Alan García del aula mag...,71263708835_2408417565837425,Zorro viejo,2019-04-19,10:08:00,zorro viejo,neutral
359,El presentador de TV recordó la reunión privad...,71263708835_10157698813173836,Zoraida Rodríguez,2019-04-20,08:57:12,zoraida rodriguez,neutral
360,"🚨 #CONFIRMADO: Erasmo Reyna, abogado de Alan G...",71263708835_10157689377373836,Zoila 😮,2019-04-17,08:00:12,zoila,neutral
361,🔴 #AHORA | El ex mandatario Ollanta Humala lle...,71263708835_10157694133878836,Zhark Camacho Rivera,2019-04-18,17:22:20,zhark camacho rivera,neutral


In [49]:
#Analizamos cuales son las cantidades agrupadadas
df_publicaciones.clasificacion.value_counts()

clasificacion
neutral     33307
negativo    13127
sarcasmo     5646
positivo     2186
Name: count, dtype: int64

In [50]:
# Guardar archivo CSV en la ruta especificada en tu computadora
df_publicaciones.to_csv('Publicaciones.csv', index=False)

print(f"Archivo guardado exitosamente como 'Publicaciones1")

Archivo guardado exitosamente como 'Publicaciones1
